In [1]:
import numpy as np
from scipy import stats
import pandas as pd
from scipy import interpolate
from scipy.signal import welch
from biosppy.signals import eda
import os, json


In [2]:
#change work directory to Code
workDir = 'D:/LAB/放鬆-三總/Relaxing/分析'
os.chdir(workDir)

In [3]:
sampleRate = 256
amp = 0.001

feature_names=['diff' ,'startle', 'duration', 
            'average_filter', 'mini_scr', 'maxi_scr', 
            'average_scr', 'std_scr', 'average_ampl',
            'std_ampl']

# lable_names = ['addictLabel', 'stateLabel', 'trainingCount']
lable_names = ['addictLabel', 'stateLabel']

data_name = ['data_name']

# testState = ['pre', 'VR', 'post']
# testDuration = [300, 480, 300]
# stagePoint = [-1080*sampleRate, -780*sampleRate, -300*sampleRate,]

inputPath = "Data3group/GSR/"
featureOutputPath = "Feature3group/"

In [4]:
def scr_calculate(raw):
    scr = eda.eda(signal=raw, sampling_rate=sampleRate, show=False, min_amplitude=amp)
    scr = list(scr)
    # print(scr)  #2維  5個一維   前2個個數相同 後3個個數相同

    ts           = scr[0]  # array   Signal time axis reference (seconds).
    filtered     = scr[1]  # array   Filtered EDA signal.
    onsets_index = scr[2]  # array   Indices of SCR pulse onsets.
    peaks_index  = scr[3]  # array   Indices of the SCR peaks.
    amplitudes   = scr[4]  # array   SCR pulse amplitudes.

    onsets = []
    peaks = []
    for o_i in onsets_index:
        onsets.append(filtered[o_i])
    for p_i in peaks_index:
        peaks.append(filtered[p_i])
    onsets = np.array(onsets)
    peaks = np.array(peaks)

    return ts, filtered, onsets, peaks, amplitudes, onsets_index, peaks_index

In [5]:
def feature_Cal(sc_list):
    # 倆倆差值
    diff_array = np.array(sc_list[1:]) - np.array(sc_list[:-1])
    diff_mean = diff_array.mean()

    # try:
    # 算SCR
    ts, filtered, onsets, peaks, amplitudes, onsets_index, peaks_index = scr_calculate(sc_list)

    # 刺激數
    startle = len(onsets)
    duration = []

    # filtered後平均
    average_filter = filtered.mean()

    # filtered後標準差
    # average_filter = np.std(filtered)

    # 平均scr持續時間?(index-index)
    for i in range(len(onsets_index)):
        duration.append(peaks_index[i] - onsets_index[i])
    duration = np.array(duration).mean()

    if len(peaks) != 0:
        # 極小 scr峰值
        mini_scr = min(peaks)
        # 極大 scr峰值
        maxi_scr = max(peaks)
        # 平均scr峰值
        average_scr = peaks.mean()
        # scr標準差
        std_scr = np.std(peaks)
    else:
        mini_scr,maxi_scr,average_scr,std_scr = np.nan, np.nan, np.nan, np.nan

    # 極小 振福
    # mini_ampl = min(amplitudes)
    # 極大 振福
    # maxi_ampl = max(amplitudes)
    # 平均振幅
    average_ampl = amplitudes.mean()
    # 振福標準差
    std_ampl = np.std(amplitudes)
    # except:
    #     diff_mean = np.nan
    #     startle = np.nan
    #     duration = np.nan
    #     average_filter = np.nan
    #     mini_scr = np.nan
    #     maxi_scr = np.nan
    #     average_scr = np.nan
    #     std_scr = np.nan
    #     average_ampl = np.nan
    #     std_ampl = np.nan

    result = {}
    result['diff'] = diff_mean
    result['startle'] = startle
    result['duration'] = duration
    result['average_filter'] = average_filter
    result['mini_scr'] = mini_scr
    result['maxi_scr'] = maxi_scr
    result['average_scr'] = average_scr
    result['std_scr'] = std_scr
    result['average_ampl'] = average_ampl
    result['std_ampl'] = std_ampl
    return  result 

In [6]:
def set_label(dataInfo):
    # addictLabel = int(dataInfo[0][0] == 'A')
    addictLabel = int(dataInfo[0][0] == 'A')

    # if len(dataInfo) == 2:
        # trainingCount = 1
        # state = dataInfo[1]
    # elif len(dataInfo) == 3:
        # trainingCount = int(dataInfo[1])
        # state = dataInfo[2]
    
    if dataInfo[3] == "PreTest":
        state = 0
    elif dataInfo[3] == "VRTest":
        state = 1
    else:
        state = 2

    stateLabel = state

    # label = {'addictLabel' : addictLabel, 'stateLabel':stateLabel, 'trainingCount': trainingCount}
    label = {'addictLabel' : addictLabel, 'stateLabel':stateLabel}
    
    return label


In [7]:
def CalGSRFeatures(filePath, sampleRate, output = False):
    
    dataList = os.listdir(filePath)
    name = pd.DataFrame(columns = data_name)
    result = pd.DataFrame(columns=feature_names)
    lable = pd.DataFrame(columns=lable_names)

    for dataName in dataList:
        # if dataName == 'A09_1.csv' or dataName == 'A09_2.csv':
        #     continue

        # N1_VR_1_PreTest_Skin
        dataInfo = dataName[:-4].split('_')
        
        data = pd.read_csv("%s%s"%(filePath, dataName), names=['TimeStamp', 'GSRdata'])
        # print(data[0:5])

        # for i in range(0, 3): #3 stages
            # if i != 2:
            #     stageData = data[stagePoint[i]:stagePoint[i+1]]
            # elif i == 2:
            #     stageData = data[stagePoint[i]:]

            # features = feature_Cal(stageData['GSRdata'])
            # name = name.append({"data_name" : "%s_%s"%(dataName[:-4], testState[i])}, ignore_index= True)
            # result = result.append(features, ignore_index=True)
            # lable = lable.append(set_label(dataInfo=dataInfo+[i]), ignore_index=True)
        features = feature_Cal(data['GSRdata'])
        # print(type(features)) #dict
        # name = name.append({"data_name" : "%s_%s"%(dataInfo[0], dataInfo[2][:-4])}, ignore_index= True)
        datanameDF = pd.DataFrame({'data_name' : ["%s%s_%s"%(dataInfo[0], dataInfo[1][0], dataInfo[3][:-4])]})
        name = pd.concat([name, datanameDF], axis=0, ignore_index= True)
        # result = result.append(features, ignore_index=True)
        result = pd.concat([result, pd.DataFrame(features, index=[0])], axis=0, ignore_index=True)
        # lable = lable.append(set_label(dataInfo), ignore_index=True)
        lable = pd.concat([lable, pd.DataFrame(set_label(dataInfo), index=[0])], axis=0, ignore_index=True)
        
    result = name.join(result)
    result = result.join(lable)


        # result = result.fillna(result.mean())
    return result

In [8]:
feature = CalGSRFeatures(inputPath, sampleRate=sampleRate, output=False)


In [9]:
feature.to_csv('Feature3group/GSR.csv')
# feature.sort_values(by=['stateLabel', 'data_name'])
feature

,data_name,diff,startle,duration,average_filter,mini_scr,maxi_scr,average_scr,std_scr,average_ampl,std_ampl,addictLabel,stateLabel
0,A100C_Post,0.000063,61,134.770492,-3583.358189,-3584.635388,-3582.542013,-3583.326907,0.571061,0.119507,0.138504,1,2
1,A100C_Pre,0.000063,54,164.796296,-3594.970028,-3602.947562,-3591.435967,-3594.891820,2.228752,0.560979,1.072982,1,0
2,A101V_Post,0.000375,29,138.620690,-3483.293143,-3496.745411,-3460.077017,-3483.226101,10.858705,1.586332,5.138217,1,2
3,A101V_Pre,-0.000031,61,125.573770,-3583.496463,-3584.515202,-3582.359135,-3583.536194,0.580260,0.110898,0.096652,1,0
4,A102C_Post,-0.000250,54,99.333333,-3539.821289,-3543.072045,-3534.301782,-3539.874022,2.144971,0.142123,0.383057,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,A97V_Pre,0.000031,58,121.913793,-3603.559400,-3604.508936,-3601.994088,-3603.595261,0.476122,0.110325,0.191659,1,0
74,A98C_Post,0.000031,62,130.693548,-3599.743775,-3600.087921,-3599.209063,-3599.737885,0.167568,0.052724,0.037903,1,2
75,A98C_Pre,0.000063,63,117.412698,-3604.840092,-3605.938200,-3604.194886,-3604.850612,0.388117,0.140205,0.129284,1,0
76,A99V_Post,0.000031,49,132.530612,-3584.026729,-3593.857332,-3582.875829,-3584.108633,1.779346,0.222968,0.586615,1,2
